# AR Aging and AR Trial Balance Recon

# 11/30/25

In [1]:
import pandas as pd
import numpy as np

aged_path = "AR_AgedInvoiceReport.xlsx"
tb_path   = "AR_TrialBalanceDetail.xlsx"

In [2]:
aged_raw = pd.read_excel(aged_path)
tb_raw   = pd.read_excel(tb_path)

/usr/local/python/3.12.1/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/python/3.12.1/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
aged_raw.head()

,Customer/_x000a_Invoice Date,Unnamed: 1,Invoice _x000a_Number,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Discount_x000a_Amount,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Days_x000a_Delq,Invoice,Discount,1 Days,31 Days,61 Days,91 Days
0,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
1,0000003,NaT,Kalispell 3rd Ave Credit Cards,NaN,NaN,NaT,NaN,NaN,Contact:,Margaret Flynn,...,NaN,NaN,NaN,Credit Limit:,0,NaN,NaT,NaN,NaN,NaN
2,NaN,NaT,2025-12-30 00:00:00,W277031-IN,NaN,2026-01-10,NaN,00:00:00,NaN,0,...,0,0,0,0,0,NaN,NaT,NaN,NaN,NaN
3,NaN,2025-12-31,6066713-PP,NaN,NaN,2025-12-31,NaN,00:00:00,NaN,0,...,0,0,0,0,0,NaN,NaT,NaN,NaN,NaN
4,NaN,2025-12-31,6066872-PP,NaN,NaN,2025-12-31,NaN,00:00:00,NaN,0,...,0,0,0,0,0,NaN,NaT,NaN,NaN,NaN


In [4]:
tb_raw.head()

,"CityServiceValcon, LLC (CSV)",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Customer/_x000a_Invoice_x000a_Number,__________Dates____________________x000a_Invoi...,NaN,NaN,Invoice Amount,Discount Amount,Invoice Balance,___________Transaction_____________x000a_Type ...,NaT,NaN,Check_x000a_Number/Type,Deposit_x000a_Date
1,0000003 Kalispell 3rd Ave Credit Cards,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN
2,6066713-PP,2025-12-31 00:00:00,2025-12-31 00:00:00,00:00:00,-1923.1,0,-1923.1,PRE,2025-12-31,-1923.10,C6066713,2025-12-31 00:00:00
3,6066872-PP,2025-12-31 00:00:00,2025-12-31 00:00:00,00:00:00,35.45,0,32,PRE,2025-12-31,35.45,C6066872,2025-12-31 00:00:00
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PAY,2025-12-31,-3.45,1225CCFEES,2025-12-31 00:00:00


In [5]:
aged = aged_raw.rename(columns={
    "Customer/_x000a_Invoice Date": "raw_customer_or_date",
    "Invoice _x000a_Number": "raw_invoice_or_name",
    "_x000a_Balance": "open_amount",
    "Unnamed: 1": "invoice_date",
})

In [6]:
cust_col = aged["raw_customer_or_date"].astype(str)

# 7-digit DM2 customer numbers (0000003, etc.)
mask_code = cust_col.str.match(r"^\d{7}$")

aged["customer_id"] = np.where(mask_code, cust_col, np.nan)
aged["customer_name"] = np.where(mask_code, aged["raw_invoice_or_name"], np.nan)

aged["customer_id"] = aged["customer_id"].ffill()
aged["customer_name"] = aged["customer_name"].ffill()

In [7]:
aged["invoice_number"] = aged["raw_invoice_or_name"]
aged["open_amount"]    = pd.to_numeric(aged["open_amount"], errors="coerce")
aged["invoice_date"]   = pd.to_datetime(aged["invoice_date"], errors="coerce")

In [8]:
aged_inv = aged[
    aged["invoice_number"].notna() &
    aged["open_amount"].notna()
].copy()

In [9]:
aged_inv.head()
aged_inv[["customer_id", "customer_name", "invoice_number", "open_amount"]].head()

,customer_id,customer_name,invoice_number,open_amount
2,0000003,Kalispell 3rd Ave Credit Cards,2025-12-30 00:00:00,3357.02
3,0000003,Kalispell 3rd Ave Credit Cards,6066713-PP,-1923.10
4,0000003,Kalispell 3rd Ave Credit Cards,6066872-PP,32.00
5,0000003,Kalispell 3rd Ave Credit Cards,W277091-IN,1306.34
9,0000005,North Lewiston Credit Cards,2025-12-30 00:00:00,347.11


In [10]:
tb = tb_raw.rename(columns={
    "CityServiceValcon, LLC (CSV)": "raw_invoice_or_cust",
    "Unnamed: 1": "invoice_date",
    "Unnamed: 6": "open_amount",
})

In [11]:
cust = tb["raw_invoice_or_cust"].astype(str)

cust_match = cust.str.extract(r"^(?P<customer_id>\d{7})\s+(?P<customer_name>.+)$")

tb["customer_id"] = cust_match["customer_id"]
tb["customer_name"] = cust_match["customer_name"]

tb["customer_id"] = tb["customer_id"].ffill()
tb["customer_name"] = tb["customer_name"].ffill()

In [12]:
tb["invoice_number"] = tb["raw_invoice_or_cust"]
tb["open_amount"]    = pd.to_numeric(tb["open_amount"], errors="coerce")
tb["invoice_date"]   = pd.to_datetime(tb["invoice_date"], errors="coerce")

/tmp/ipykernel_2225/2413210351.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  tb["invoice_date"]   = pd.to_datetime(tb["invoice_date"], errors="coerce")


In [13]:
tb_inv = tb[tb["open_amount"].notna()].copy()

# Drop "Customer 0000003 Totals:" etc.
tb_inv = tb_inv[~tb_inv["invoice_number"].astype(str).str.startswith("Customer")]

# Drop the customer header rows like "0000003 Kalispell 3rd Ave..."
tb_inv = tb_inv[~tb_inv["invoice_number"].astype(str).str.match(r"^\d{7}\s+")]

# Drop the grand "Report Totals:" line
tb_inv = tb_inv[~tb_inv["invoice_number"].astype(str).str.contains("Report Totals:", na=False)]

In [14]:
tb_inv.head()
tb_inv[["customer_id", "customer_name", "invoice_number", "open_amount"]].head()

,customer_id,customer_name,invoice_number,open_amount
2,0000003,Kalispell 3rd Ave Credit Cards,6066713-PP,-1923.10
3,0000003,Kalispell 3rd Ave Credit Cards,6066872-PP,32.00
5,0000003,Kalispell 3rd Ave Credit Cards,W277031-IN,3357.02
6,0000003,Kalispell 3rd Ave Credit Cards,W277091-IN,1306.34
9,0000005,North Lewiston Credit Cards,6067521-PP,-104.08


In [15]:
aged_total = aged_inv["open_amount"].sum()
tb_total   = tb_inv["open_amount"].sum()

aged_total, tb_total, aged_total - tb_total

(np.float64(18643431.53),
 np.float64(18643323.98),
 np.float64(107.55000000074506))

In [16]:
inv_recon = aged_inv.merge(
    tb_inv[["customer_id", "invoice_number", "open_amount"]],
    on=["customer_id", "invoice_number"],
    how="outer",
    suffixes=("_aged", "_tb"),
)

inv_recon["open_amount_aged"] = inv_recon["open_amount_aged"].fillna(0)
inv_recon["open_amount_tb"]   = inv_recon["open_amount_tb"].fillna(0)

inv_recon["variance"] = inv_recon["open_amount_aged"] - inv_recon["open_amount_tb"]

In [17]:
inv_issues = inv_recon[inv_recon["variance"].round(2) != 0]

inv_issues.sort_values(["customer_id", "invoice_number"]).head(50)

,raw_customer_or_date,invoice_date,raw_invoice_or_name,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Discount_x000a_Amount,...,Discount,1 Days,31 Days,61 Days,91 Days,customer_id,customer_name,invoice_number,open_amount_tb,variance
0,NaN,NaT,2025-12-30 00:00:00,W277031-IN,NaN,2026-01-10,NaN,00:00:00,NaN,0,...,NaN,NaT,NaN,NaN,NaN,0000003,Kalispell 3rd Ave Credit Cards,2025-12-30 00:00:00,0.00,3357.02
3,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,0000003,NaN,W277031-IN,3357.02,-3357.02
5,NaN,NaT,2025-12-30 00:00:00,W277033-IN,NaN,2026-01-10,NaN,00:00:00,NaN,0,...,NaN,NaT,NaN,NaN,NaN,0000005,North Lewiston Credit Cards,2025-12-30 00:00:00,0.00,347.11
7,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,0000005,NaN,W277033-IN,347.11,-347.11
9,NaN,NaT,2025-12-30 00:00:00,0906522-IN,NaN,2026-01-10,NaN,00:00:00,NaN,0,...,NaN,NaT,NaN,NaN,NaN,0000006,Kalispell 5th Ave Credit Cards,2025-12-30 00:00:00,0.00,3465.17
10,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,0000006,NaN,0906522-IN,3465.17,-3465.17
16,NaN,NaT,2025-12-30 00:00:00,0906523-IN,NaN,2026-01-10,NaN,00:00:00,NaN,0,...,NaN,NaT,NaN,NaN,NaN,0000011,Butte CHS Credit Cards,2025-12-30 00:00:00,0.00,6983.26
17,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,0000011,NaN,0906523-IN,6983.26,-6983.26
20,NaN,NaT,2025-12-30 00:00:00,0906526-IN,NaN,2026-01-10,NaN,00:00:00,NaN,0,...,NaN,NaT,NaN,NaN,NaN,0000013,St. Maries Credit Cards,2025-12-30 00:00:00,0.00,3863.73
21,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,0000013,NaN,0906526-IN,3863.73,-3863.73


In [18]:
aged_cust = (
    aged_inv
    .groupby(["customer_id", "customer_name"], as_index=False)["open_amount"]
    .sum()
    .rename(columns={"open_amount": "aged_open_amount"})
)

tb_cust = (
    tb_inv
    .groupby(["customer_id", "customer_name"], as_index=False)["open_amount"]
    .sum()
    .rename(columns={"open_amount": "tb_open_amount"})
)

cust_recon = aged_cust.merge(
    tb_cust,
    on=["customer_id", "customer_name"],
    how="outer"
)

cust_recon[["aged_open_amount", "tb_open_amount"]] = cust_recon[["aged_open_amount", "tb_open_amount"]].fillna(0)
cust_recon["variance"] = cust_recon["aged_open_amount"] - cust_recon["tb_open_amount"]

cust_issues = cust_recon[cust_recon["variance"].round(2) != 0].sort_values("variance")
cust_issues.head(20)

,customer_id,customer_name,aged_open_amount,tb_open_amount,variance
7399,0098457,Thelma Bear Medicine,215.1,107.55,107.55


In [19]:
with pd.ExcelWriter("AR_Recon_Workpapers.xlsx", engine="openpyxl") as writer:
    inv_recon.to_excel(writer, sheet_name="Invoice_Recon_All", index=False)
    inv_issues.to_excel(writer, sheet_name="Invoice_Issues", index=False)
    cust_recon.to_excel(writer, sheet_name="Customer_Recon_All", index=False)
    cust_issues.to_excel(writer, sheet_name="Customer_Issues", index=False)